# 计算BOF和TDA所需要的时间

## 一个计算运行时间的函数

In [1]:
import time

def function_A(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function {func.__name__} took {execution_time} seconds to execute.")
        return result
    return wrapper

# 示例函数B
def function_B(n):
    # 这里可以是任何你想要计算时间的函数
    for i in range(n):
        _ = i * i

# 使用function_A包装function_B
timed_function_B = function_A(function_B)

# 调用timed_function_B，并传入参数
timed_function_B(1000000)  # 传入函数B所需的参数


Function function_B took 0.035030364990234375 seconds to execute.
